In [1]:
import pyodbc
import json
from tqdm import tqdm_notebook
import xlrd
from xlwt import Workbook, Formula

In [2]:
#Creating the connection (local)
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=localhost\SQLEXPRESS;'
                      'Database=pollution;'
                      'Trusted_Connection=yes;')

In [3]:
#This function insert into an Excel sheet the result of a request
#Input : the request and the name of the Excel sheet
#Output : /
def insert_excel(request, name_sheet):
    sheet = workbook.add_sheet(name_sheet)
    select_request = conn.cursor()
    select_request.execute(request)
    result = []
    for line in select_request:
        result.append(line)
    n = len(result[0])
    for j in range(n):
        for i in range(len(result)):
            sheet.write(i, j, result[i][j])
    sheet.write(0, 6, request)

In [4]:
#Names of sheet give in Excel
name_sheets = ['req1', 'req2', 'req3', 'req4', 'req5', \
               
               'req6', 'req7', 'req8', 'req9', 'req10', \
               'req11', 'req12', 'req13', 'req14', 'req15', \
               'req16', 'req17', 'req18', 'req19', 'req20']

In [5]:
#This list contain all ours requests
list_request = ["Select TOP 30 label_word, sum(tf) \
                from belong b, word w \
                where w.id_word = b.id_word \
                and id_article <> 542 \
                and label_word not in ('000') \
                group by label_word \
                order by 2 desc", \
                
               "select TOP 30 sum(tf), label_word, id_article \
                from belong b, word w \
                where b.id_word = w.id_word \
                and id_article <> 542 \
                and b.is_country = 1 \
                group by label_word, id_article \
                order by 1 desc", \

               "select TOP 30 sum(tf), label_word, name_newspaper \
                from belong b, word w, article a, newspaper n  \
                where b.id_word = w.id_word \
                and a.id_article = b.id_article \
                and a.id_newspaper = n .id_newspaper \
                and b.id_article <> 542 \
                and label_word not in ('000') \
                group by label_word, name_newspaper \
                order by 1 desc", \
                
               "select TOP 30 sum(tf), label_word, left(publication_date,4) \
                from belong b, word w, article a \
                where b.id_word = w.id_word \
                and a.id_article = b.id_article \
                and b.id_article <> 542 \
                and b.is_country = 1 \
                and left(publication_date,4)  between '1950' and '2019' \
                group by label_word, left(publication_date,4) \
                order by 1 desc", \
                
               "select count(label_word), left(publication_date, 4) \
                from belong b, word w, article a \
                where b.id_word = w.id_word \
                and a.id_article = b.id_article \
                and a.id_article <> 542 \
                group by left(publication_date, 4) \
                order by 1 desc", \
                
               "select label_word, a.id_article, tf, publication_date \
                from belong b, word w, article a \
                where b.id_word = w.id_word \
                and a.id_article = b.id_article \
                and a.id_article <> 542 \
                and label_word in ('ocean', 'plastiques', 'plastique', 'mer', 'dechets', 'dechet', 'eau', 'pollution', \
                'microplastique') \
                order by 4 desc", \
                
               "select distinct label_word, left(publication_date, 7), \
                case when  left(publication_date, 7) between '2010-01' and '2019-03' THEN 1 \
                when  left(publication_date, 7) between '2000-01' and '2010-01' THEN 2 \
                when  left(publication_date, 7) between '1990-01' and '2000-01' THEN 3 \
                when  left(publication_date, 7) between '1980-01' and '1990-01' THEN 4 \
                when  left(publication_date, 7) between '1970-01' and '1980-01' THEN 5 \
                when  left(publication_date, 7) between '1960-01' and '1970-01' THEN 6 \
                when  left(publication_date, 4) between '1950-01' and '1960-01' THEN 7 \
                ELSE 8 END \
                from belong b, word w, article a \
                where b.id_word = w.id_word \
                and b.id_article = a.id_article \
                and b.id_article <> 542 \
                and is_country = 1 \
                order by 2 desc" ,\
                
                "select distinct label_word, left(publication_date, 4), \
                case when  left(publication_date, 4) between '2010' and '2019' THEN 1 \
                when  left(publication_date, 4) between '2000' and '2010' THEN 2 \
                when  left(publication_date, 4) between '1990' and '2000' THEN 3 \
                when  left(publication_date, 4) between '1980' and '1990' THEN 4 \
                when  left(publication_date, 4) between '1970' and '1980' THEN 5 \
                when  left(publication_date, 4) between '1960' and '1970' THEN 6 \
                when  left(publication_date, 4) between '1950' and '1960' THEN 7 \
                ELSE 8 END \
                from belong b, word w, article a \
                where b.id_word = w.id_word \
                and b.id_article = a.id_article \
                and b.id_article <> 542 \
                and is_country = 1 \
                order by 2 desc" ,\
                
                "select a.word, a.date_pub, \
                case when  left(a.date_pub, 7) between '2010-01' and '2019-03' THEN 1 \
                when  left(a.date_pub, 7) between '2000-01' and '2010-01' THEN 2 \
                when  left(a.date_pub, 7) between '1990-01' and '2000-01' THEN 3 \
                when  left(a.date_pub, 7) between '1980-01' and '1990-01' THEN 4 \
                when  left(a.date_pub, 7) between '1970-01' and '1980-01' THEN 5 \
                when  left(a.date_pub, 7) between '1960-01' and '1970-01' THEN 6 \
                when  left(a.date_pub, 7) between '1950-01' and '1960-01' THEN 7 \
                ELSE 8 END \
                from \
                ( \
                select TOP 30 count(distinct label_word) as word, left(publication_date, 7) as date_pub \
                from belong b, word w, article a \
                where b.id_word = w.id_word \
                and b.id_article = a.id_article \
                and b.id_article <> 542 \
                and is_country = 1 \
                group by left(publication_date, 7) \
                order by 2 desc \
                ) a"]      

list_request.append("select a.word, a.date_pub, \
                case when  left(a.date_pub, 4) between '2010' and '2019' THEN 1 \
                when  left(a.date_pub, 4) between '2000' and '2010' THEN 2 \
                when  left(a.date_pub, 4) between '1990' and '2000' THEN 3 \
                when  left(a.date_pub, 4) between '1980' and '1990' THEN 4 \
                when  left(a.date_pub, 4) between '1970' and '1980' THEN 5 \
                when  left(a.date_pub, 4) between '1960' and '1970' THEN 6 \
                when  left(a.date_pub, 4) between '1950' and '1960' THEN 7 \
                ELSE 8 END \
                from \
                ( \
                select TOP 30 count(distinct label_word) as word, left(publication_date, 4) as date_pub \
                from belong b, word w, article a \
                where b.id_word = w.id_word \
                and b.id_article = a.id_article \
                and b.id_article <> 542 \
                and is_country = 1 \
                group by left(publication_date, 4) \
                order by 1 desc \
                ) a")

list_request.append("Select TOP 8 count(b.id_word), label_word \
                from belong b, word w \
                where w.id_word = b.id_word \
                and label_word not in ('ete', 'ans') \
                and b.is_description = 1 \
                group by label_word \
                order by 1 desc")
                
list_request.append("Select TOP 8 count(b.id_word), label_word \
                from belong b, word w \
                where w.id_word = b.id_word \
                and label_word not in ('ete', 'ans') \
                and b.is_description = 1 \
                group by label_word \
                order by 1 desc")
                
list_request.append("Select TOP 8 count(b.id_word), label_word \
                from belong b, word w \
                where w.id_word = b.id_word \
                and label_word not in ('ete', 'ans') \
                and b.is_title = 1 \
                group by label_word \
                order by 1 desc")

list_request.append("Select TOP 8 count(b.id_word), label_word \
                from belong b, word w \
                where w.id_word = b.id_word \
                and label_word not in ('ete', 'ans') \
                and b.is_title = 1 \
                and b.is_description = 1 \
                group by label_word \
                order by 1 desc")

In [6]:
#Insert into excel
workbook = Workbook()
for i in range(len(list_request)):
    request = list_request[i]
    name_sheet = name_sheets[i]
    insert_excel(request, name_sheet)
workbook.save("../database/all_result_request.xls")

# <center> FIN </center>